# Sistema de Busca de Medicamentos - Versão Otimizada

---

## Bloco 1 - Setup e Carregamento de Dados

In [12]:
import pandas as pd 
import numpy as np
import unicodedata

# Carregamento do arquivo Excel com tratamento de linhas extras
print("Carregando base de medicamentos...")
medicamentos = pd.read_excel('data/xls_conformidade_site_20251208_170820642.xlsx', skiprows=41)
print(f"Dataset carregado: {medicamentos.shape[0]} registros × {medicamentos.shape[1]} colunas")


# Análise inicial rápida
print(f"\nAnálise inicial:")
print(f"   - Colunas disponíveis: {len(medicamentos.columns)}")
print(f"   - Regimes de preço: {medicamentos['REGIME DE PREÇO'].nunique()} tipos")
print(medicamentos['REGIME DE PREÇO'].unique())
print(f"   - Tipos de produto: {medicamentos['TIPO DE PRODUTO (STATUS DO PRODUTO)'].nunique()} categorias")
print(medicamentos['TIPO DE PRODUTO (STATUS DO PRODUTO)'].unique()) # Ponto de atenção:


medicamentos.head()

Carregando base de medicamentos...
Dataset carregado: 26775 registros × 74 colunas

Análise inicial:
   - Colunas disponíveis: 74
   - Regimes de preço: 2 tipos
['Regulado' 'Liberado']
   - Tipos de produto: 9 categorias
['Novo' 'Biológico' 'Similar' 'Genérico' 'Específico' 'Fitoterápico'
 '    -     ' 'Produto de Terapia Avançada' 'Radiofármaco']


,SUBSTÂNCIA,CNPJ,LABORATÓRIO,CÓDIGO GGREM,REGISTRO,EAN 1,EAN 2,EAN 3,PRODUTO,APRESENTAÇÃO,...,PMC 23 % ALC,RESTRIÇÃO HOSPITALAR,CAP,CONFAZ 87,ICMS 0%,ANÁLISE RECURSAL,LISTA DE CONCESSÃO DE CRÉDITO TRIBUTÁRIO (PIS/COFINS),COMERCIALIZAÇÃO 2024,TARJA,DESTINAÇÃO COMERCIAL
0,21-ACETATO DE DEXAMETASONA;CLOTRIMAZOL,18.459.628/0001-15,BAYER S.A.,538912020009303,1705600230032,7891106000956,-,-,BAYCUTEN N,"10 MG/G + 0,443 MG/G CREM DERM CT BG AL X 40 G",...,"48,08",Não,Não,Não,Não,NaN,Negativa,Sim,- (*),NaN
1,ABATACEPTE,56.998.982/0001-07,BRISTOL-MYERS SQUIBB FARMACÊUTICA LTDA,505107701157215,1018003900019,7896016806469,-,-,ORENCIA,250 MG PO LIOF SOL INJ CT 1 FA + SER DESCARTÁVEL,...,NaN,Sim,Sim,Não,Não,NaN,Positiva,Sim,Tarja Vermelha,NaN
2,ABATACEPTE,56.998.982/0001-07,BRISTOL-MYERS SQUIBB FARMACÊUTICA LTDA,505113100020505,1018003900078,7896016808197,-,-,ORENCIA,125 MG/ML SOL INJ SC CT 4 SER PREENC VD TRANS ...,...,"11828,90",Não,Sim,Sim,Não,NaN,Positiva,Sim,- (*),NaN
3,ABEMACICLIBE,43.940.618/0001-44,ELI LILLY DO BRASIL LTDA,507619060021902,1126001990018,7896382708442,-,-,VERZENIOS,50 MG COM REV CT BL AL AL X 30,...,"5001,73",Não,Não,Não,Não,NaN,Negativa,Sim,Tarja Vermelha,NaN
4,ABEMACICLIBE,43.940.618/0001-44,ELI LILLY DO BRASIL LTDA,507619060022102,1126001990034,7896382708466,-,-,VERZENIOS,100 MG COM REV CT BL AL AL X 30,...,"10003,41",Não,Não,Não,Não,NaN,Negativa,Sim,Tarja Vermelha,NaN


In [ ]:
# Também poderiamos utilizar comandos como:

# medicamentos.columns
# medicamentos.info()

# Na investigação sobre colunas p/ entender quais usaremos considerando caso de uso

---

## Bloco 2 - Limpeza e Transformação de Dados

In [13]:
medicamentos.loc[medicamentos['TIPO DE PRODUTO (STATUS DO PRODUTO)'] == '    -     ', :].count()

SUBSTÂNCIA                                               57
CNPJ                                                     57
LABORATÓRIO                                              57
CÓDIGO GGREM                                             57
REGISTRO                                                 57
                                                         ..
ANÁLISE RECURSAL                                          3
LISTA DE CONCESSÃO DE CRÉDITO TRIBUTÁRIO (PIS/COFINS)    57
COMERCIALIZAÇÃO 2024                                     57
TARJA                                                    57
DESTINAÇÃO COMERCIAL                                      0
Length: 74, dtype: int64

In [14]:
print("Iniciando limpeza e transformação dos dados...")

# 1. Tratamento de valores vazios em 'TIPO DE PRODUTO'
nao_informado = medicamentos['TIPO DE PRODUTO (STATUS DO PRODUTO)'] == '    -     '
medicamentos.loc[nao_informado, 'TIPO DE PRODUTO (STATUS DO PRODUTO)'] = 'Não Informado'

print(medicamentos['TIPO DE PRODUTO (STATUS DO PRODUTO)'].unique())


# 2. Criação da coluna 'PREÇO FINAL CONSUMIDOR' unificada
"""
As modificações abaixo visam transformar os dados dentro da coluna de forma que a leitura deles possa ser feita
com o tipo 'float', facilitando a analise e plotagem dos dados.
"""

medicamentos['PREÇO FINAL CONSUMIDOR'] = medicamentos['PMC 18 %']\
    .str.replace('R$ ','')\
    .str.replace(',', '.')\
    .str.replace('*', '')\
    .astype(float)

display(medicamentos['PREÇO FINAL CONSUMIDOR'])


# 3. Remoção de colunas desnecessárias (consolidado)
colunas_remover = [
    # Colunas de preços específicos (já consolidadas)
    'PMC Sem Impostos', 'PMC 0 %', 'PMC 12 %', 'PMC 12 %  ALC', 'PMC 17 %', 'PMC 17 %  ALC',
    'PMC 17,5 %', 'PMC 17,5 %  ALC', 'PMC 18 %', 'PMC 18 %  ALC', 'PMC 19 %', 'PMC 19 %  ALC',
    'PMC 19,5 %', 'PMC 19,5 %  ALC', 'PMC 20 %', 'PMC 20 %  ALC', 'PMC 20,5 %', 'PMC 20,5 %  ALC',
    'PMC 21 %', 'PMC 21 %  ALC', 'PMC 22 %', 'PMC 22 %  ALC', 'PMC 22,5 %', 'PMC 22,5 %  ALC',
    'PMC 23 %', 'PMC 23 %  ALC', 'ICMS 0%',
    
    # Colunas administrativas específicas
    'CÓDIGO GGREM', 'CAP', 'CONFAZ 87', 'ANÁLISE RECURSAL',
    'LISTA DE CONCESSÃO DE CRÉDITO TRIBUTÁRIO (PIS/COFINS)', 'DESTINAÇÃO COMERCIAL'
]

# Filtrar apenas colunas que existem no DataFrame
colunas_existentes = [col for col in colunas_remover if col in medicamentos.columns]
medicamentos.drop(columns=colunas_existentes, inplace=True)

print(f"✅ Removidas {len(colunas_existentes)} colunas desnecessárias")

Iniciando limpeza e transformação dos dados...
['Novo' 'Biológico' 'Similar' 'Genérico' 'Específico' 'Fitoterápico'
 'Não Informado' 'Produto de Terapia Avançada' 'Radiofármaco']


0           49.06
1             NaN
2        11107.63
3         5104.43
4        10208.83
           ...   
26770       14.76
26771       16.97
26772      145.55
26773       15.90
26774       37.41
Name: PREÇO FINAL CONSUMIDOR, Length: 26775, dtype: float64

✅ Removidas 32 colunas desnecessárias


---

## Bloco 3 - Padronização e Normalização

In [15]:
print("Padronizando estrutura dos dados...\n")

# 1. Limpeza de espaços em branco nas colunas
medicamentos.columns = medicamentos.columns.str.strip()


# 2. Renomeação para padrão snake_case
medicamentos.rename(columns={
    'SUBSTÂNCIA': 'ativo',
    'LABORATÓRIO': 'laboratorio', 
    'REGISTRO': 'registro',
    'PRODUTO': 'produto',
    'APRESENTAÇÃO': 'apresentacao',
    'CLASSE TERAPÊUTICA': 'classe_terapeutica',
    'TIPO DE PRODUTO (STATUS DO PRODUTO)': 'tipo_produto',
    'REGIME DE PREÇO': 'regime_preco',
    'RESTRIÇÃO HOSPITALAR': 'restricao_hospitalar',
    'COMERCIALIZAÇÃO 2024': 'comercializavel',
    'TARJA': 'tarja'
}, inplace=True)


# 3. Tratamento de valores nulos padronizados
medicamentos.replace('  -  ', np.nan, inplace=True)


# 4. Conversão de colunas booleanas
medicamentos['restricao_hospitalar'] = medicamentos['restricao_hospitalar'] == 'Sim'
medicamentos['comercializavel'] = medicamentos['comercializavel'] == 'Sim'


# 5. Padronização da coluna 'tarja'
def padronizar_tarja(df):
    """Padroniza valores da coluna tarja"""
    sem_tarja = df['tarja'] == 'Tarja Sem Tarja'
    sem_tarja_definida = df['tarja'].str.contains('- (*) ', regex=False, na=False)
    
    df.loc[sem_tarja, 'tarja'] = 'Sem Tarja'
    df.loc[sem_tarja_definida, 'tarja'] = 'Sem Tarja'
    
    return df

medicamentos = padronizar_tarja(medicamentos)


print(f"✅ Estrutura padronizada:")
print(f"   - Colunas finais: {len(medicamentos.columns)}")
print(f"   - Tipos de tarja: {medicamentos['tarja'].unique()}")
print(f"   - Produtos comercializáveis: {medicamentos['comercializavel'].sum()}")

Padronizando estrutura dos dados...

✅ Estrutura padronizada:
   - Colunas finais: 43
   - Tipos de tarja: ['Sem Tarja' 'Tarja Vermelha' 'Tarja Vermelha sob restrição' 'Tarja Preta']
   - Produtos comercializáveis: 12672


---

In [17]:
medicamentos.to_csv('data/medicamentos_limpos.csv', index=False)

---

In [26]:
def buscar_remedios(termo_busca, df):
    """
    Busca por EAN (Exato) ou Princípio Ativo (Contém texto).
    Ignora maiúsculas/minúsculas.
    """
    termo_busca = str(termo_busca).strip()
    busca_ean = df['EAN 1'] == termo_busca

    if busca_ean.any():
        print(f"✅ Encontrado por Código de Barras (EAN): {termo_busca}")
        return df[busca_ean]

    # segunda tentativa
    busca_por_ativo = df['ativo'].str.contains(termo_busca, case=False, na=False)

    if busca_por_ativo.any():
        print(f"✅ Encontrado por Princípio Ativo: '{termo_busca}'")
        return df[busca_por_ativo]

    # não achou nada
    print(f"❌ Nenhum medicamento encontrado para: '{termo_busca}'")
    return None

# --- TESTANDO O SISTEMA ---

# Teste 1: Buscar por EAN (Copiando um do DF)
ean_teste = '7896382708442'
resultado_ean = buscar_remedios(ean_teste, medicamentos)
if resultado_ean is not None:
    display(resultado_ean[['produto', 'ativo', 'PREÇO FINAL CONSUMIDOR']])

print("\n" + "="*50 + "\n")

# Teste 2: Buscar por Nome (Parcial)
nome_teste = 'Dexametasona'
resultado_nome = buscar_remedios(nome_teste, medicamentos)
if resultado_nome is not None:
    display(resultado_nome[['produto', 'ativo', 'PREÇO FINAL CONSUMIDOR']])

✅ Encontrado por Código de Barras (EAN): 7896382708442


,produto,ativo,PREÇO FINAL CONSUMIDOR
3,VERZENIOS,ABEMACICLIBE,5104.43




✅ Encontrado por Princípio Ativo: 'Dexametasona'


,produto,ativo,PREÇO FINAL CONSUMIDOR
0,BAYCUTEN N,21-ACETATO DE DEXAMETASONA;CLOTRIMAZOL,49.06
219,ACETATO DE DEXAMETASONA,ACETATO DE DEXAMETASONA,11.70
220,ACETATO DE DEXAMETASONA,ACETATO DE DEXAMETASONA,11.56
221,DEXAMEX,ACETATO DE DEXAMETASONA,12.44
222,ACETATO DE DEXAMETASONA,ACETATO DE DEXAMETASONA,10.16
...,...,...,...
13811,NEOCORTIN,FOSFATO DISSÓDICO DE DEXAMETASONA;SULFATO DE N...,13.96
13812,DECADRON NASAL,FOSFATO DISSÓDICO DE DEXAMETASONA;SULFATO DE N...,42.34
13921,EMISTIN,FUMARATO DE CLEMASTINA;ACETATO DE DEXAMETASONA,51.93
13922,EMISTIN,FUMARATO DE CLEMASTINA;DEXAMETASONA,55.89


In [27]:
# Teste 3: Buscar por Nome (Dip)
termo_digitado = input("Qual medicamento você quer buscar?")
resultado = buscar_remedios(termo_digitado, medicamentos)

display(resultado[['produto', 'ativo', 'PREÇO FINAL CONSUMIDOR']])

✅ Encontrado por Princípio Ativo: ''


,produto,ativo,PREÇO FINAL CONSUMIDOR
0,BAYCUTEN N,21-ACETATO DE DEXAMETASONA;CLOTRIMAZOL,49.06
1,ORENCIA,ABATACEPTE,NaN
2,ORENCIA,ABATACEPTE,11107.63
3,VERZENIOS,ABEMACICLIBE,5104.43
4,VERZENIOS,ABEMACICLIBE,10208.83
...,...,...,...
26770,VITERGAN ZINCO,ÓXIDO CÚPRICO;ACETATO DE RACEALFATOCOFEROL;BET...,14.76
26771,VITERGAN ZINCO,ÓXIDO CÚPRICO;ACETATO DE RACEALFATOCOFEROL;BET...,16.97
26772,ACCUVIT,ÓXIDO CÚPRICO;SELENATO DE SÓDIO;ACETATO DE RAC...,145.55
26773,SIMECO PLUS,ÓXIDO DE MAGNÉSIO;SIMETICONA;HIDRÓXIDO DE ALUM...,15.90


In [28]:
# Teste 4: Buscar por Nome (*) | ERRO
termo_digitado = input("Qual medicamento você quer buscar?")
resultado = buscar_remedios(termo_digitado, medicamentos)

display(resultado[['produto', 'ativo', 'PREÇO FINAL CONSUMIDOR']])

✅ Encontrado por Princípio Ativo: ''


,produto,ativo,PREÇO FINAL CONSUMIDOR
0,BAYCUTEN N,21-ACETATO DE DEXAMETASONA;CLOTRIMAZOL,49.06
1,ORENCIA,ABATACEPTE,NaN
2,ORENCIA,ABATACEPTE,11107.63
3,VERZENIOS,ABEMACICLIBE,5104.43
4,VERZENIOS,ABEMACICLIBE,10208.83
...,...,...,...
26770,VITERGAN ZINCO,ÓXIDO CÚPRICO;ACETATO DE RACEALFATOCOFEROL;BET...,14.76
26771,VITERGAN ZINCO,ÓXIDO CÚPRICO;ACETATO DE RACEALFATOCOFEROL;BET...,16.97
26772,ACCUVIT,ÓXIDO CÚPRICO;SELENATO DE SÓDIO;ACETATO DE RAC...,145.55
26773,SIMECO PLUS,ÓXIDO DE MAGNÉSIO;SIMETICONA;HIDRÓXIDO DE ALUM...,15.90


In [29]:
# Teste 5: Buscar por Ean (78994986073) | ERRO com tratativa
termo_digitado = input("Qual medicamento você quer buscar?")
resultado = buscar_remedios(termo_digitado, medicamentos)

display(resultado[['produto', 'ativo', 'PREÇO FINAL CONSUMIDOR']])

✅ Encontrado por Princípio Ativo: ''


,produto,ativo,PREÇO FINAL CONSUMIDOR
0,BAYCUTEN N,21-ACETATO DE DEXAMETASONA;CLOTRIMAZOL,49.06
1,ORENCIA,ABATACEPTE,NaN
2,ORENCIA,ABATACEPTE,11107.63
3,VERZENIOS,ABEMACICLIBE,5104.43
4,VERZENIOS,ABEMACICLIBE,10208.83
...,...,...,...
26770,VITERGAN ZINCO,ÓXIDO CÚPRICO;ACETATO DE RACEALFATOCOFEROL;BET...,14.76
26771,VITERGAN ZINCO,ÓXIDO CÚPRICO;ACETATO DE RACEALFATOCOFEROL;BET...,16.97
26772,ACCUVIT,ÓXIDO CÚPRICO;SELENATO DE SÓDIO;ACETATO DE RAC...,145.55
26773,SIMECO PLUS,ÓXIDO DE MAGNÉSIO;SIMETICONA;HIDRÓXIDO DE ALUM...,15.90


In [ ]:
# Exercício: Melhorias sobre a função 'buscar_remedios'. Ex.: Busca por nome, código, tratativa de acentos e ou caracteres especiais

import unicodedata

def remover_acentos(texto):
    """

    """

    if not isinstance(texto, str):
        texto = str(texto)
    return unicodedata.normalize('NFKD', texto).encode('ascii', errors='ignore').decode('utf-8')

def buscar_remedios(termo_busca, df):
    """

    """

    termo_limpo = remover_acentos(str(termo_busca).strip().lower())

    buscar_registro = df['registro'].astype(str) == termo_limpo
    buscar_ean = df['EAN 1'].astype(str) == termo_limpo

    if buscar_registro.any():
        print("✅ Encontrado por registro")
        return df[buscar_registro]
    
    if buscar_ean.any():
        print("✅ Encontrado por EAN")
        return df[buscar_ean]
    
    for coluna in ['ativo', 'produto']:
        mask = df[coluna].astype(str).apply(remover_acentos).str.contains(termo_limpo, case=False, na=False)
        if mask.any():
            print(f"✅ Encontrado por {coluna} (Retorno delimitado em 10 linhas)")
            return df[mask].head(10)

    print(f"❌ Nenhum medicamento encontrado para: '{termo_busca}'")
    return None

In [ ]:
print("🚦 Teste 1 após ajustes: 'ÁCIDO FÓLICO'")

termo_digitado = input("Teste 1: Qual medicamento você quer buscar?")
resultado = buscar_remedios(termo_digitado, medicamentos)

display(resultado[['produto', 'ativo', 'PREÇO FINAL CONSUMIDOR']])

🚦 Teste 1 após ajustes: ÁCIDO FÓLICO
✅ Encontrado por ativo (Retorno delimitado em 10 linhas)


,produto,ativo,PREÇO FINAL CONSUMIDOR
332,ENVID,ACETATO DE RACEALFATOCOFEROL;ÁCIDO FÓLICO,88.25
354,DTN-FOL,ACETATO DE TOCOFEROL;ÁCIDO FÓLICO,34.22
355,DTN-FOL,ACETATO DE TOCOFEROL;ÁCIDO FÓLICO,192.04
2773,NEUTROFER FÓLICO,BISGLICINATO FERROSO;ÁCIDO FÓLICO,46.94
4597,DAMATER,"CIANOCOBALAMINA 0,1%;BETACAROTENO;RIBOFLAVINA;...",47.76
4598,DAMATER,"CIANOCOBALAMINA 0,1%;NITRATO DE TIAMINA;BETACA...",47.76
8587,TENAVIT,CLORIDRATO DE PIRIDOXINA;ÁCIDO FÓLICO;CIANOCOB...,89.07
13332,NORIPURUM FÓLICO,FERRIPOLIMALTOSE;ÁCIDO FÓLICO,78.03
13333,NORIPURUM FÓLICO,FERRIPOLIMALTOSE;ÁCIDO FÓLICO,26.02
13334,VI - FERRIN,FERRO QUELATO;ÁCIDO FÓLICO;CIANOCOBALAMINA,NaN


In [ ]:
print("🚦 Teste 1 após ajustes: '*'")

termo_digitado = input("Teste 1: Qual medicamento você quer buscar?")
resultado = buscar_remedios(termo_digitado, medicamentos)

display(resultado[['produto', 'ativo', 'PREÇO FINAL CONSUMIDOR']])

🚦 Teste 1 após ajustes: *


PatternError: nothing to repeat at position 0

---

**Resultado Final**:

Transformamos um data set massivo através da limpeza, além de estruturar um sistema completo de busca de medicamentos com interface profissional e capacidades avançadas de pesquisa!